In [1]:
import time
import datetime
import board
import adafruit_dht
import adafruit_bmp280
from gpiozero import LED, Button
from transformers import pipeline

In [2]:
# Initialize sensors and actuators
DHT22Sensor = adafruit_dht.DHT22(board.D16)
i2c = board.I2C()
bmp280Sensor = adafruit_bmp280.Adafruit_BMP280_I2C(i2c, address=0x76)
bmp280Sensor.sea_level_pressure = 1013.25

# Initialize LEDs and Button
ledRed = LED(13)
ledYlw = LED(19)
ledGrn = LED(26)
button = Button(20)

In [ ]:
# Initialize the SLM pipeline
# We're using a small model suitable for Raspberry Pi
generator = pipeline('text-generation', 
                    model='TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T',
                    device='cpu')

In [ ]:
def get_sensor_data():
    """Get current readings from all sensors"""
    try:
        temp_dht = DHT22Sensor.temperature
        humidity = DHT22Sensor.humidity
        temp_bmp = bmp280Sensor.temperature
        pressure = bmp280Sensor.pressure
        
        return {
            'temperature_dht': round(temp_dht, 1) if temp_dht else None,
            'humidity': round(humidity, 1) if humidity else None,
            'temperature_bmp': round(temp_bmp, 1),
            'pressure': round(pressure, 1)
        }
    except RuntimeError:
        return None

def control_leds(red=False, yellow=False, green=False):
    """Control LED states"""
    ledRed.value = red
    ledYlw.value = yellow
    ledGrn.value = green

In [ ]:
def generate_response(sensor_data):
    """Generate response based on sensor data using SLM"""
    if not sensor_data:
        return "Unable to read sensor data"
    
    prompt = f"""Based on these sensor readings:
    Temperature: {sensor_data['temperature_dht']}°C
    Humidity: {sensor_data['humidity']}%
    Pressure: {sensor_data['pressure']} hPa
    
    Provide a brief status and recommendation in 2 sentences.
    """
    
    # Generate response from SLM
    response = generator(prompt, 
                       max_length=100,
                       num_return_sequences=1,
                       temperature=0.7)[0]['generated_text']
    
    return response

def process_conditions(sensor_data):
    """Process sensor data and control LEDs based on conditions"""
    if not sensor_data:
        control_leds(red=True)  # Error condition
        return
    
    temp = sensor_data['temperature_dht']
    humidity = sensor_data['humidity']
    
    # Example conditions for LED control
    if temp > 30:  # Hot
        control_leds(red=True)
    elif humidity > 70:  # Humid
        control_leds(yellow=True)
    else:  # Normal conditions
        control_leds(green=True)

In [5]:
def main_loop():
    """Main program loop"""
    print("Starting Physical Computing with SLM Integration...")
    print("Press the button to get a reading and SLM response.")
    
    try:
        while True:
            if button.is_pressed:
                # Get sensor readings
                sensor_data = get_sensor_data()
                
                # Process conditions and control LEDs
                process_conditions(sensor_data)
                
                if sensor_data:
                    # Get SLM response
                    response = generate_response(sensor_data)
                    
                    # Print current status
                    print("\nCurrent Readings:")
                    print(f"Temperature: {sensor_data['temperature_dht']}°C")
                    print(f"Humidity: {sensor_data['humidity']}%")
                    print(f"Pressure: {sensor_data['pressure']} hPa")
                    print("\nSLM Response:")
                    print(response)
                    
                time.sleep(2)  # Debounce and allow time to read
            
            time.sleep(0.1)  # Reduce CPU usage
            
    except KeyboardInterrupt:
        print("\nShutting down...")
        control_leds(False, False, False)  # Turn off all LEDs
        

In [ ]:
main_loop()

Starting Physical Computing with SLM Integration...
Press the button to get a reading and SLM response.


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/mjrovai/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Current Readings:
Temperature: 26.6°C
Humidity: 42.0%
Pressure: 907.4 hPa

SLM Response:
Based on these sensor readings:
    Temperature: 26.6°C
    Humidity: 42.0%
    Pressure: 907.4 hPa
    
    Provide a brief status and recommendation in 2 sentences.
    
    The temperature is a bit high, but the humidity is low.
    
    The temperature is a bit high, but the humidity is low.


In [ ]:
if __name__ == "__main__":
    main_loop()